# IMDB Sentiment-Analyse: Naive Bayes vs. LSTM
Dieses Notebook lädt das IMDB-Dataset aus dem Ordner `Data/` und trainiert zwei Klassifikatoren: einen klassischen Ansatz (TF-IDF + Naive Bayes) und ein Deep-Learning-Modell (LSTM). Zum Schluss vergleichen wir Metriken, Laufzeit und Interpretierbarkeit und geben Empfehlungen.

Hinweis: Die Zellen sind so gestaltet, dass das Notebook auf einem typischen Studenten-Laptop lauffähig ist (begrenzte Vokabulargröße, wenige Epochen).

In [ ]:
# Section 1: Imports und Einstellungen
import os
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
import joblib
import nltk
from nltk.corpus import stopwords
import re
# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Reproduzierbarkeit
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Zeige Versionen
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('tensorflow', tf.__version__)

In [ ]:
# Section 2: Daten laden (robustes Snippet)
data_path = os.path.join('Data', 'IMDB Dataset.csv')
if not os.path.exists(data_path):
    raise FileNotFoundError(f'Datei nicht gefunden: {data_path} - stelle sicher, dass sie im Ordner Data/ liegt')

df = pd.read_csv(data_path)
print('Shape:', df.shape)
display(df.head(5))
display(df.info())

In [ ]:
# Section 3: EDA - Klassenverteilung & Längenverteilung
if 'sentiment' in df.columns and 'review' in df.columns:
    display(df['sentiment'].value_counts())
    sns.countplot(data=df, x='sentiment')
    plt.title('Class distribution')
    plt.show()
    df['review_len'] = df['review'].apply(lambda x: len(str(x).split()))
    plt.figure(figsize=(8,4))
    sns.histplot(df['review_len'], bins=50)
    plt.title('Review length distribution (words)')
    plt.show()
else:
    print('Erwarte Spalten `review` und `sentiment` im Dataset')

In [ ]:
# Section 4: Preprocessing - clean_text Funktion
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text, remove_stopwords=True):
    if not isinstance(text, str):
        text = str(text)
    # HTML entfernen
    text = re.sub(r'<.*?>', ' ', text)
    # non-letters entfernen
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    tokens = text.split()
    if remove_stopwords:
        tokens = [t for t in tokens if t not in STOPWORDS]
    return ' '.join(tokens)

# Sample Anwendung (erst nur auf kleinen Subset um Zeit zu sparen)
df['clean_review'] = df['review'].astype(str).str[:500].apply(clean_text)  # safe apply
display(df[['review','clean_review']].head())

In [ ]:
# Label-Encoding und train/test split (Section 4/6)
label_map = {'positive':1, 'negative':0}
df['label'] = df['sentiment'].map(label_map)
X = df['clean_review']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
print('Train/Test sizes:', X_train.shape, X_test.shape)

### Modell 1: TF-IDF + Naive Bayes (klassischer Ansatz)
Wir verwenden `TfidfVectorizer` + `MultinomialNB`. Dieser Ansatz ist schnell, gut interpretierbar (Top-Features) und für Bag-of-Words-Textklassifikation häufig sehr effektiv.

In [ ]:
# Utility: Metriken-Funktion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}

def plot_confusion(y_true, y_pred, labels=[0,1], title='Confusion Matrix'):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.show()

In [ ]:
# Section 6: Train Naive Bayes
nb_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1,2))),
    ('nb', MultinomialNB())
])
t0 = time.time()
nb_pipeline.fit(X_train, y_train)
t1 = time.time()
print(f'Training Naive Bayes took {t1-t0:.2f} sec')
y_pred_nb = nb_pipeline.predict(X_test)
metrics_nb = compute_metrics(y_test, y_pred_nb)
print('Naive Bayes results:', metrics_nb)
print('
Classification report:
', classification_report(y_test, y_pred_nb))
plot_confusion(y_test, y_pred_nb, title='Naive Bayes Confusion Matrix')
# Speichere Modell
joblib.dump(nb_pipeline, 'nb_pipeline.joblib')
print('NB pipeline saved to nb_pipeline.joblib')

### Modell 2: LSTM (Deep Learning)
Wir erstellen ein einfaches LSTM-Modell mit Embedding-Layer. Längere Trainingszeit und höhere Rechenkosten können auftreten; deshalb begrenzen wir Vokabulargröße und Epochen.

In [ ]:
# Section 7: Tokenisierung und Sequenzvorbereitung
MAX_NUM_WORDS = 20000
MAX_LEN = 200
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')
print('Vocabulary size (approx):', min(MAX_NUM_WORDS, len(tokenizer.word_index)))
print('X_train_pad shape:', X_train_pad.shape)

In [ ]:
# Section 8: LSTM Modellaufbau und Training
EMBEDDING_DIM = 128
model = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# Callbacks
es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
# train (keeping epochs small for student machines)
t0 = time.time()
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=128, validation_split=0.1, callbacks=[es], verbose=1)
t1 = time.time()
print(f'LSTM training took {t1-t0:.2f} sec')
# Speichere Modell und Tokenizer
model.save('lstm_model.h5')
joblib.dump(tokenizer, 'tokenizer.joblib')
print('LSTM model and tokenizer saved')

In [ ]:
# Section 10: Evaluation LSTM
y_probs = model.predict(X_test_pad, batch_size=128)
y_pred_lstm = (y_probs.flatten() >= 0.5).astype(int)
metrics_lstm = compute_metrics(y_test, y_pred_lstm)
print('LSTM results:', metrics_lstm)
print('
Classification report:
', classification_report(y_test, y_pred_lstm))
plot_confusion(y_test, y_pred_lstm, title='LSTM Confusion Matrix')
# Plot Training History
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(); plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend(); plt.title('Accuracy')
plt.show()

In [ ]:
# Section 11: Vergleich beider Modelle in einer Tabelle
results = pd.DataFrame([
    {'model':'NaiveBayes', **metrics_nb},
    {'model':'LSTM', **metrics_lstm}
])
display(results)
results.to_csv('model_comparison.csv', index=False)
print('Vergleich gespeichert als model_comparison.csv')

### Interpretation & Diskussion (kurz)
- Naive Bayes ist sehr schnell, interpretiert durch Top-Features (TF-IDF Gewichtungen).
- LSTM kann Kontext und Reihenfolge lernen, ist aber rechenaufwändiger und benötigt mehr Daten oder Regularisierung, um nicht zu überfitten.
- Wenn NB ähnliche oder bessere Metriken liefert, ist der klassische Ansatz für Produktions-Einsätze oft ausreichend; andernfalls lohnt sich Fein-Tuning oder der Einsatz vortrainierter Embeddings/Transformer.

In [ ]:
# Section 12: Top-Features aus NB anzeigen (Interpretierbarkeit)
tfidf = nb_pipeline.named_steps['tfidf']
nb = nb_pipeline.named_steps['nb']
feature_names = tfidf.get_feature_names_out()
class0_top = np.argsort(nb.feature_log_prob_[0])[-20:]
class1_top = np.argsort(nb.feature_log_prob_[1])[-20:]
print('Top features negative:')
print(feature_names[class0_top])
print('Top features positive:')
print(feature_names[class1_top])

### Reproduzierbarkeit & Hinzufügungen
- Modelle: `nb_pipeline.joblib`, `lstm_model.h5`, `tokenizer.joblib` wurden gespeichert.
- Paketversionen oben ausgeben; für vollständige Reproduzierbarkeit siehe `requirements.txt`.
- Für weitergehende Verbesserungen: Hyperparameter-Tuning (GridSearchCV für NB-Pipeline), pretrained embeddings (GloVe), oder Transformer-Modelle (z.B. DistilBERT).

## How to run
1. Installiere Abhängigkeiten: `pip install -r requirements.txt`
2. Öffne dieses Notebook in VS Code oder Jupyter und führe Zellen nacheinander aus.
3. Falls NLTK-Stopwords nicht vorhanden sind, wird der Download in einer Zelle ausgeführt.